In [1]:
from pathlib import Path
from utils import read_input
from collections import Counter, deque
import numpy as np
from bresenham import bresenham
from statistics import mean, median, mode
from itertools import combinations, product, pairwise, permutations
from tqdm import tqdm
import pandas as pd
from rich import print
import matplotlib.pyplot as plt

In [2]:
test = Path("input/day15/test.txt")
data = Path("input/day15/data.txt")

In [3]:
def risk_map(inputfile):
    raw = read_input(inputfile)
    return [[int(num) for num in line] for line in raw]

In [4]:
def min_risk(risk_grid):
    rows = len(risk_grid)
    cols = len(risk_grid[0])
    target_row, target_col = rows - 1, cols - 1
    
    # initialise grid
    total_risk = np.zeros((rows, cols))
    
    # risk for starting pos is 0
    total_risk[0][0] = risk_grid[0][0]
    
    # cumulative risk for path through first col
    for i in range(1, target_row + 1):
        total_risk[i][0] = total_risk[i-1][0] + risk_grid[i][0]
 
    # cumulative risk for path through first row
    for j in range(1, target_col + 1):
        total_risk[0][j] = total_risk[0][j-1] + risk_grid[0][j]

    # min cumulative risk for path through rest of array based on min risk of previous cells (up and left only)
    for i in range(1, target_row + 1):
        for j in range(1, target_col + 1):
            total_risk[i][j] = min(total_risk[i-1][j], total_risk[i][j-1]) + risk_grid[i][j]
    return total_risk[target_row][target_col] - risk_grid[0][0]

In [5]:
grid = risk_map(test)
print(grid)
min_risk(grid)

[
    [1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
    [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
    [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
    [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
    [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
    [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
    [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
    [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
    [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
    [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]
]

40.0

In [6]:
grid = risk_map(data)
min_risk(grid)

506.0

In [68]:

def get_neighbours(node, rows, cols, visited):
    nbs = []
    r, c = node
    for row, col in [(r - 1, c), (r + 1, c), (r, c -1), (r, c + 1)]:
        if 0 <= row < rows:
            if 0 <= col < cols:
                if visited[row][col] == 0:
                    yield row, col
        
        
def dijkstra(cost_grid):
    rows = len(cost_grid)
    cols = len(cost_grid[0])
    
    visited = np.zeros((rows, cols))
    
    shortest = np.full((rows, cols), np.inf)
    shortest[0][0] = 0
    
    previous_nodes = {}
    
    finished = False
    while not finished:
        # find shortest, not visited node
        masked = np.ma.array(shortest, mask=visited)
        shortest_value = masked.min()
    
        shortest_node_indices = np.where(shortest == shortest_value)
        shortest_nodes = list(zip(*np.where(shortest == shortest_value)))
        shortest_nodes = [node for node in shortest_nodes if visited[node[0]][node[1]] == 0]
            
        shortest_node = shortest_nodes[0]

        # update visited
        visited[shortest_node[0]][shortest_node[1]] = 1

        # find neighbours that are not already visited
        neighbours = get_neighbours(shortest_node, rows, cols, visited)
        # find their costs from cost_grid (cost of current node + cost of neighbour) and update shortest if it is smaller

        for neighbour in neighbours:
            cost = shortest[shortest_node[0]][shortest_node[1]] + cost_grid[neighbour[0]][neighbour[1]]
            current_cost = shortest[neighbour[0]][neighbour[1]]
            if cost < current_cost:
                shortest[neighbour[0]][neighbour[1]] = cost
                previous_nodes[neighbour] = shortest_node
        
        finished = visited.all()
    return shortest, previous_nodes
        
    

In [96]:
def find_min_cost(grid, shortest, previous_nodes):
    start_node = (0, 0)
    target_node = (len(grid) - 1,len(grid[0]) - 1)

    path = []
    current_node = target_node

    while current_node != start_node:
        path.append(current_node)
        current_node = previous_nodes[current_node]

    # Add the start node manually if included
    # path.append(node)

    cost = 0
    for r, c in path:
        cost += grid[r][c]
    return cost

In [97]:
grid = risk_map(data)
shortest, previous_nodes = dijkstra(grid)

In [98]:
find_min_cost(grid, shortest, previous_nodes)

498

In [127]:
def extend_grid(grid):
    grids = [grid]

    for i in range(1, 5):
        grid = repeat_grid(grid)
        grids.append(grid)
    row = np.hstack(grids)

    rows = [row]
    for i in range(1, 5):
        row = repeat_grid(row)
        rows.append(row)

    new_grid = np.vstack(rows)
    return new_grid

In [130]:
grid = np.array(risk_map(test))
new_grid = extend_grid(grid)
shortest, previous_nodes = dijkstra(new_grid)
find_min_cost(new_grid, shortest, previous_nodes)

315

In [131]:
grid = np.array(risk_map(data))
new_grid = extend_grid(grid)
shortest, previous_nodes = dijkstra(new_grid)
find_min_cost(new_grid, shortest, previous_nodes)

2901